<img src="https://raw.githubusercontent.com/paular143/images_notebooks/master/logo-sin-nombre.png" alt="Universidad de Los Andes" style="width: 90px;" align="left" frameborder="300"/>

**UNIVERSIDAD DE LOS ANDES** <br>
**FACULTAD DE ECONOMÍA - EDUCACIÓN CONTINUADA** <br>
**Análisis espacial de datos y sus aplicaciones en Python** <br>
**PROFESORES:** Germán González <br><br>


# Clase 3: Estadísticas descriptivas de geolocalización

- Paquete _geopandas_.
- Capas y siluetas. 




In [1]:
# !pip3 install geopandas
# !pip3 install shapely
# !pip3 install numpy
# !pip3 install folium
# !pip3 install geojson
# !pip3 install branca

from shapely import geometry, ops
import numpy as np
import shapely.geometry as shg
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
import random
import pandas as pd
import geopandas as gpd
import folium
import json
import geojson
from shapely.ops import cascaded_union
import branca
import re
import unicodedata


/Users/germangonzalez/.pyenv/versions/3.7.5/envs/motus/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## COVID - Cruce de bases de datos

## Lectura de casos de COVID

In [3]:
Casos_COVID  = pd.read_excel('Datos/COVID.xlsx')

## Lectura de un archivo SHP

In [4]:
File= 'Datos/MPM/MPM_FuenteCensal_2018.shp'
Data_census = gpd.read_file(File)
#### ---  EPSG:4326 -> WGS84 ---- ## 
Data_census.to_crs(epsg=4326, inplace=True)


In [5]:
Data_census.columns

Index(['DPTO_CCDGO', 'MPIO_CCDGO', 'MPIO_CNMBR', 'MPIO_CCNCT', 'DPTO_CNMBR',
       'MPM', 'MPM_CAB', 'MPM_CPRD', 'AFBMO_TOT', 'AFBMO_CAB', 'AFBMO_CPRD',
       'BLE_TOT', 'BLE_CAB', 'BLE_CPRD', 'BSCPI_TOT', 'BSCPI_CAB',
       'BSCPI_CPRD', 'BASS_TOT', 'BASS_CAB', 'BASS_CPRD', 'TDE_TOT', 'TDE_CAB',
       'TDE_CPRD', 'HC_TOT', 'HC_CAB', 'HC_CPRD', 'IEE_TOT', 'IEE_CAB',
       'IEE_CPRD', 'IE_TOT', 'IE_CAB', 'IE_CPRD', 'MIPE_TOT', 'MIPE_CAB',
       'MIPE_CPRD', 'MIP_TOT', 'MIP_CAB', 'MIP_CPRD', 'RE_TOT', 'RE_CAB',
       'RE_CPRD', 'SAFAM_TOT', 'SAFAM_CAB', 'SAFAM_CPRD', 'SAS_TOT', 'SAS_CAB',
       'SAS_CPRD', 'TRINF_TOT', 'TRINF_CAB', 'TRINF_CPRD', 'TRIML_TOT',
       'TRIML_CAB', 'TRIML_CPRD', 'geometry'],
      dtype='object')

## Este archivo contiene 52 variables sobre los municipios de colombia 

## Seleccionaremos solo tres variables de todas estas
* **MPM:**  Medida de pobreza multidimensional de fuente censal en el total del municipio
* **AFBMO_TOT:**  Indicador de analfabetismo en el total del municipio
* **TRIML_TOT:**  Indicador de trabajo informal en el total del municipio

In [6]:
Data_census = Data_census[['MPIO_CCNCT','DPTO_CNMBR','MPIO_CNMBR','MPM','AFBMO_TOT','TRIML_TOT','geometry']]

In [7]:
Data_census.head()

,MPIO_CCNCT,DPTO_CNMBR,MPIO_CNMBR,MPM,AFBMO_TOT,TRIML_TOT,geometry
0,18001,CAQUETÁ,FLORENCIA,29.6,9.1,85.0,"POLYGON ((-75.42074 2.19413, -75.42042 2.19412..."
1,18029,CAQUETÁ,ALBANIA,38.8,12.1,88.1,"POLYGON ((-75.89506 1.36570, -75.89506 1.36570..."
2,18094,CAQUETÁ,BELÉN DE LOS ANDAQUÍES,50.0,18.4,88.6,"POLYGON ((-75.78705 1.74982, -75.78715 1.74956..."
3,18247,CAQUETÁ,EL DONCELLO,44.3,14.7,90.7,"POLYGON ((-75.36167 2.32143, -75.36142 2.32122..."
4,18256,CAQUETÁ,EL PAUJÍL,45.5,14.7,90.0,"POLYGON ((-75.36638 2.20952, -75.35963 2.17388..."


## Renombramos las variables

In [8]:
Data_census.columns = ['Codigo','Departamento','Municipio','Pobreza','Analfabestimo','Trabajo informal','Geometria']
### ----- Volvemos el código un String ----- ##### 
Data_census['Codigo']= Data_census['Codigo'].apply(lambda x: str(x))
Data_census=Data_census.set_index('Codigo')

In [9]:
def Arreglar_tilde(Texto):
    Texto = unicodedata.normalize('NFD', Texto)
    Texto = Texto.encode('ascii', 'ignore')
    Texto = Texto.decode("utf-8")
    return(Texto)

In [10]:
###### -------- Para Departamento ------- ######
Data_census['Departamento'] = Data_census['Departamento'].apply(lambda x: Arreglar_tilde(Texto=x))
###### -------- Para Municipio ------- ######

Data_census['Municipio'] = Data_census['Municipio'].apply(lambda x: Arreglar_tilde(Texto=x))

In [11]:
Casos_COVID.head()

,ID de caso,Fecha de notificación,Codigo DIVIPOLA,Ciudad de ubicación,Departamento o Distrito,atención,Edad,Sexo,Tipo,Estado,País de procedencia,FIS,Fecha de muerte,Fecha diagnostico,Fecha recuperado,fecha reporte web,Tipo recuperación
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,Italia,2020-02-27T00:00:00.000,NaN,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,España,2020-03-04T00:00:00.000,NaN,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR
2,3,2020-03-07T00:00:00.000,5001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,España,2020-02-29T00:00:00.000,NaN,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,PCR
3,4,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,Colombia,2020-03-06T00:00:00.000,NaN,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,PCR
4,5,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,Colombia,2020-03-08T00:00:00.000,NaN,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,PCR


## Ahora nos concetramos en la base de datos de casos del COVID


In [12]:
### ---- Filtramos la base de datos --- ### 
Casos_COVID=Casos_COVID[['Codigo DIVIPOLA','ID de caso']]
Casos_COVID.columns = ['Codigo','ID']
### ----- Volvemos el código un String ----- ##### 
Casos_COVID['Codigo']= Casos_COVID['Codigo'].apply(lambda x: str(x))

## Cuántos casos existen por código

In [13]:
Casos_COVID=Casos_COVID.groupby('Codigo').count()
### --- Cambiamos el nombre ---- ####
Casos_COVID.columns = ['Casos Covid']

## Se consolidan las bases

In [14]:
Casos_COVID = pd.concat([Casos_COVID,Data_census],axis = 1)
# --- Se devuele el nombre a los indices -- ##
Casos_COVID = Casos_COVID.rename_axis('Codigo')
# --- Se devuele los indices -- ##
Casos_COVID = Casos_COVID.reset_index(drop=False)

In [15]:
#### ------ Se remplazan Nan por cero ----- #### 
Casos_COVID['Casos Covid'] = Casos_COVID['Casos Covid'].fillna(0)
Casos_COVID = Casos_COVID.dropna()

## Cómo se relacionan estas variables

In [16]:
Casos_COVID[['Casos Covid','Pobreza','Analfabestimo','Trabajo informal']].corr()

,Casos Covid,Pobreza,Analfabestimo,Trabajo informal
Casos Covid,1.000000,-0.085593,-0.091096,-0.098760
Pobreza,-0.085593,1.000000,0.796599,0.738982
Analfabestimo,-0.091096,0.796599,1.000000,0.627045
Trabajo informal,-0.098760,0.738982,0.627045,1.000000


In [17]:
### ---- Se crea una llave ---- ### 

In [18]:
Casos_COVID['Str_code'] = Casos_COVID['Municipio'] + ' - ' + Casos_COVID['Codigo']

## Construcción de un Geojson que contega el número de casos de COVID por unidad geográfica 

In [19]:
#### -- Creamos una lista vacia ---- #### 
Figuras_json = []

##### ----- Para cada una de las figuras se va consolidar un polígono ------ ##### 
for k in range(0,len(Casos_COVID)):
    Zona_analisis = geojson.Feature(geometry=Casos_COVID.iloc[k]['Geometria'])

    Poligono = {
        'type': 'Feature',
        'geometry': { "type": Zona_analisis['geometry']['type'], "coordinates": Zona_analisis['geometry']['coordinates']},
        'properties' :  {"id": k,"Municipio": str(Casos_COVID.iloc[k]['Str_code']) } 
    }
    #### ---- A figuras se añade este poligono ---- ### 
    Figuras_json.append(Poligono)


    
##### --------- Se consolidan todos los polígonos como una colección de datos ------ ######  
Coleccion_json = {
    'type': 'FeatureCollection',
    'features': Figuras_json,
}

### ----  Verifica que la estructura sea correcta para JSON ----- #### 
geo_str = json.dumps(Coleccion_json) 

#### ---- load ----- ##### 
geo_str = json.loads(geo_str)


#outfile = open('Covid.json', 'w+')
#outfile.write(geo_str)


## Heatmap 

In [20]:
mapa = folium.Map(location=[4.316107698094774,-74.18107270162268],zoom_start=7,tiles='cartodbpositron')
### --- Scale HeatMap --- ###

    
### --- Diferents tile formats --- ### 
folium.TileLayer('Stamen Terrain').add_to(mapa)
folium.TileLayer('openstreetmap').add_to(mapa)
folium.TileLayer('Stamen Toner').add_to(mapa)


mapa.choropleth(
 geo_data=geo_str,
 name='choropleth',
 data=Casos_COVID,
 columns=['Str_code', 'Casos Covid'],
 key_on='properties.Municipio',
 fill_color='YlGn',
 fill_opacity=0.4,
 line_opacity=0.6,
 legend_name='Covid'
)
folium.LayerControl().add_to(mapa)


mapa.save('Outputs/Covid_municipios.html')

/Users/germangonzalez/.pyenv/versions/3.7.5/envs/motus/lib/python3.7/site-packages/folium/folium.py:413: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


## Se crea un Score para el color

In [21]:
Casos_COVID['Color'] = (Casos_COVID['Casos Covid']/Casos_COVID['Casos Covid'].max())*100

## Mapa

In [22]:
mapa = folium.Map(location=[4.316107698094774,-74.18107270162268],zoom_start=7,tiles='cartodbpositron')
### --- Scale HeatMap --- ###
#### https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3 
colormap = branca.colormap.LinearColormap(colors=['#543005','#8c510a','#bf812d','#dfc27d','#f6e8c3','#f5f5f5','#c7eae5','#80cdc1','#35978f','#01665e','#003c30'], index=[0,10,20,30,40,50,60,70,80,90,100],vmin=0,vmax=100)
### --- Add Scale heatmap color --- ### 
colormap.caption = 'Casos Covid'


    
### --- Diferents tile formats --- ### 
folium.TileLayer('Stamen Terrain').add_to(mapa)
folium.TileLayer('openstreetmap').add_to(mapa)
folium.TileLayer('Stamen Toner').add_to(mapa)


folium.GeoJson(geo_str,
    style_function=lambda feature: {
        'fillColor': colormap(int(Casos_COVID.set_index('Str_code').loc[str(re.sub(pattern='Municipio:.',repl='',string=feature['properties']['Municipio']))]['Color'])),
        'color': '#788691',
        'weight': 0.5,
        'fillOpacity':0.5,
    },
    highlight_function=lambda x: {'weight':1,'fillColor':'#e7e3d7','color': '#e7e3d7'}, tooltip=folium.GeoJsonTooltip(fields=['Municipio'], labels=False,sticky=False),
    name='AE1').add_to(mapa)


colormap.add_to(mapa)


folium.LayerControl().add_to(mapa)
mapa.save('Outputs/Covid_municipios_v2.html')


In [23]:
Data_census = Data_census.reset_index(drop=False)

## Agregación por Departamentos

In [24]:
Data_depto = Casos_COVID.groupby('Departamento').apply(lambda x: pd.DataFrame({'Codigo':[x['Codigo']],'Geomety': [cascaded_union(x['Geometria']).buffer(0)] , 
                                                                              'Casos': [x['Casos Covid'].sum()]}))
Data_depto=Data_depto.reset_index(drop=False)

In [25]:
Data_depto=Data_depto.drop('level_1',1)

In [26]:
#### -- Creamos una lista vacia ---- #### 
Figuras_json_depto = []

##### ----- Para cada una de las figuras se va consolidar un polígono ------ ##### 
for k in range(0,len(Data_depto)):
    Zona_analisis = geojson.Feature(geometry=Data_depto.iloc[k]['Geomety'])

    Poligono = {
        'type': 'Feature',
        'geometry': { "type": Zona_analisis['geometry']['type'], "coordinates": Zona_analisis['geometry']['coordinates']},
        'properties' :  {"id": k,"Departamento": str(Data_depto.iloc[k]['Departamento']) } 
    }
    #### ---- A figuras se añade este poligono ---- ### 
    Figuras_json_depto.append(Poligono)


    
##### --------- Se consolidan todos los polígonos como una colección de datos ------ ######  
Coleccion_json_depto = {
    'type': 'FeatureCollection',
    'features': Figuras_json_depto,
}

### ----  Verifica que la estructura sea correcta para JSON ----- #### 
geo_str_depto = json.dumps(Coleccion_json_depto) 

#### ---- load ----- ##### 
geo_str_depto = json.loads(geo_str_depto)


#outfile = open('Covid.json', 'w+')
#outfile.write(geo_str)



In [27]:
Data_depto['Color'] = (Data_depto['Casos']/Data_depto['Casos'].max())*100

In [28]:
mapa = folium.Map(location=[4.316107698094774,-74.18107270162268],zoom_start=5,tiles='cartodbpositron')
### --- Scale HeatMap --- ###
#### https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3 
colormap = branca.colormap.LinearColormap(colors=['#ffffff','#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6','#2171b5','#08519c','#08306b','#14305a'], index=[0,10,20,30,40,50,60,70,80,90,100],vmin=0,vmax=100)
### --- Add Scale heatmap color --- ### 
colormap.caption = 'Casos Covid'


    
### --- Diferents tile formats --- ### 
folium.TileLayer('Stamen Terrain').add_to(mapa)
folium.TileLayer('openstreetmap').add_to(mapa)
folium.TileLayer('Stamen Toner').add_to(mapa)


folium.GeoJson(geo_str_depto,
    style_function=lambda feature: {
        'fillColor': colormap(int(Data_depto.set_index('Departamento').loc[str(re.sub(pattern='Departamento:.',repl='',string=feature['properties']['Departamento']))]['Color'])),
        'color': '#788691',
        'weight': 0.5,
        'fillOpacity':0.9,
    },
    highlight_function=lambda x: {'weight':1,'fillColor':'#e7e3d7','color': '#e7e3d7'}, tooltip=folium.GeoJsonTooltip(fields=['Departamento'], labels=False,sticky=False),
    name='AE1').add_to(mapa)


colormap.add_to(mapa)


folium.LayerControl().add_to(mapa)
mapa.save('Outputs/Covid_departamento.html')

